In [40]:
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete import TabularCPD

In [41]:
model = BayesianNetwork([
    ("killsDiff", "blueGoldDiff"),
    ("minionsDiff", "blueGoldDiff"),
    ("minionsDiff", "blueExperienceDiff"),
    ("towersDiff", "blueGoldDiff"),
    ("towersDiff", "blueWins"),
    ("blueHeralds", "towersDiff"),
    ("redHeralds", "towersDiff"),
    ("dragonsDiff", "blueWins"),
    ("blueExperienceDiff", "blueWins"),
    ("blueGoldDiff", "blueWins")
])

In [42]:
import pandas as pd
import numpy as np
def create_dataset(path="high_diamond_ranked_10min.csv"):
    columns = ["blueWins", "blueGoldDiff", "blueExperienceDiff", "blueHeralds", "redHeralds"]
    data = pd.read_csv(path)
    dataset = data.loc[:, columns]
    dataset["killsDiff"] = data.apply(lambda row: row["blueKills"]-row["redKills"], axis=1)
    dataset["minionsDiff"] = data.apply(lambda row: row["blueTotalMinionsKilled"]-row["redTotalMinionsKilled"], axis=1)
    dataset["dragonsDiff"] = data.apply(lambda row: row["blueDragons"]-row["redDragons"], axis=1)
    dataset["towersDiff"] = data.apply(lambda row: row["blueTowersDestroyed"]-row["redTowersDestroyed"], axis=1)
    return dataset

In [43]:
def discretize(dataset):
    dataset["killsDiff"] = pd.cut(dataset["killsDiff"], bins=[-np.inf, -2, 2, np.inf], labels=["Negative", "Neutral", "Positive"], include_lowest = True)
    dataset["minionsDiff"] = pd.cut(dataset["minionsDiff"], bins=[-np.inf, -15, +15, np.inf], labels=["Negative", "Neutral", "Positive"], include_lowest = True)
    dataset["dragonsDiff"] = pd.cut(dataset["dragonsDiff"], bins=[-np.inf, -1, 0, np.inf], labels=["Negative", "Neutral", "Positive"], include_lowest = True)
    dataset["towersDiff"] = pd.cut(dataset["towersDiff"], bins=[-np.inf, -1, 0, np.inf], labels=["Negative", "Neutral", "Positive"], include_lowest = True)
    dataset["blueGoldDiff"] = pd.cut(dataset["blueGoldDiff"], bins=[-np.inf, -1000, 1000, np.inf], labels=["Negative", "Neutral", "Positive"], include_lowest = True)
    dataset["blueExperienceDiff"] = pd.cut(dataset["blueExperienceDiff"], bins=[-np.inf, -500, 500, np.inf], labels=["Negative", "Neutral", "Positive"], include_lowest = True)

    dataset["blueHeralds"] = pd.cut(dataset["blueHeralds"], bins=[-1,0,np.inf], labels=["Low", "High"], include_lowest = True)
    dataset["redHeralds"] = pd.cut(dataset["redHeralds"], bins=[-1,0,np.inf], labels=["Low", "High"], include_lowest = True)

    return dataset

In [44]:
dataset = create_dataset()
dataset = discretize(dataset)
dataset.head()

for column in dataset:
    print(pd.value_counts(dataset[column]))



0    4949
1    4930
Name: blueWins, dtype: int64
Positive    3341
Negative    3290
Neutral     3248
Name: blueGoldDiff, dtype: int64
Negative    3986
Positive    3783
Neutral     2110
Name: blueExperienceDiff, dtype: int64
Low     8022
High    1857
Name: blueHeralds, dtype: int64
Low     8298
High    1581
Name: redHeralds, dtype: int64
Neutral     3671
Negative    3491
Positive    2717
Name: killsDiff, dtype: int64
Neutral     3769
Negative    3189
Positive    2921
Name: minionsDiff, dtype: int64
Negative    4081
Positive    3576
Neutral     2222
Name: dragonsDiff, dtype: int64
Neutral     9064
Positive     441
Negative     374
Name: towersDiff, dtype: int64


In [45]:
from pgmpy.estimators import BayesianEstimator, MaximumLikelihoodEstimator
from IPython.display import display, HTML

display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

model.cpds = []

model.fit(data=dataset,
            estimator=BayesianEstimator,
            prior_type="BDeu",
            equivalent_sample_size=10,
            complete_samples_only=False,
)
pd.options.display.max_columns = 2000

print(f"Check model: {model.check_model()=}")

for cpd in model.get_cpds():
    print(f"CPT of {cpd.variable}")
    print(cpd)



Check model: model.check_model()=True
CPT of killsDiff
+---------------------+----------+
| killsDiff(Negative) | 0.353356 |
+---------------------+----------+
| killsDiff(Neutral)  | 0.371558 |
+---------------------+----------+
| killsDiff(Positive) | 0.275087 |
+---------------------+----------+
CPT of blueGoldDiff
+------------------------+-----+-----------------------+
| killsDiff              | ... | killsDiff(Positive)   |
+------------------------+-----+-----------------------+
| minionsDiff            | ... | minionsDiff(Positive) |
+------------------------+-----+-----------------------+
| towersDiff             | ... | towersDiff(Positive)  |
+------------------------+-----+-----------------------+
| blueGoldDiff(Negative) | ... | 0.0006806888571234089 |
+------------------------+-----+-----------------------+
| blueGoldDiff(Neutral)  | ... | 0.0006806888571234089 |
+------------------------+-----+-----------------------+
| blueGoldDiff(Positive) | ... | 0.9986386222857532  

In [46]:
from pgmpy.inference import VariableElimination

model_inference = VariableElimination(model)

#print(model_inference.query(["blueWins"], {"blueGoldDiff": "Positive", "minionsDiff": "Negative"}))
print("__________________________________________________________")
#print(model_inference.query(["blueWins"], {"blueGoldDiff": "Positive", "minionsDiff": "Negative", "killsDiff": "Negative"}))
print("__________________________________________________________")
print(model_inference.query(["blueWins"], {"blueGoldDiff": "Positive", "minionsDiff": "Negative", "killsDiff": "Negative", "towersDiff": "Negative"}))
print("__________________________________________________________")
print(model_inference.query(["blueWins"], {"blueGoldDiff": "Positive", "minionsDiff": "Negative", "killsDiff": "Positive", "towersDiff": "Negative"}))
print(model_inference.query(["blueWins"], {"blueGoldDiff": "Positive", "minionsDiff": "Negative", "killsDiff": "Positive", "towersDiff": "Positive"}))
print(model_inference.query(["blueWins"], {"blueGoldDiff": "Positive", "minionsDiff": "Positive", "killsDiff": "Positive", "towersDiff": "Positive"}))




__________________________________________________________
__________________________________________________________
+-------------+-----------------+
| blueWins    |   phi(blueWins) |
+=============+=================+
| blueWins(0) |          0.4525 |
+-------------+-----------------+
| blueWins(1) |          0.5475 |
+-------------+-----------------+
__________________________________________________________
+-------------+-----------------+
| blueWins    |   phi(blueWins) |
+=============+=================+
| blueWins(0) |          0.4525 |
+-------------+-----------------+
| blueWins(1) |          0.5475 |
+-------------+-----------------+
+-------------+-----------------+
| blueWins    |   phi(blueWins) |
+=============+=================+
| blueWins(0) |          0.3368 |
+-------------+-----------------+
| blueWins(1) |          0.6632 |
+-------------+-----------------+
+-------------+-----------------+
| blueWins    |   phi(blueWins) |
+=============+=================+
| blueW

In [52]:
print(model_inference.query(["blueGoldDiff"], {"blueWins": 1, "towersDiff": "Positive"}))

+------------------------+---------------------+
| blueGoldDiff           |   phi(blueGoldDiff) |
+========================+=====================+
| blueGoldDiff(Negative) |              0.0299 |
+------------------------+---------------------+
| blueGoldDiff(Neutral)  |              0.2657 |
+------------------------+---------------------+
| blueGoldDiff(Positive) |              0.7045 |
+------------------------+---------------------+


In [58]:
def active_trails_of(query, evidence):
    active = model.active_trail_nodes(query, observed=evidence).get(query)
    active.remove(query)
    if active:
        if evidence:
            print(f'Active trails between \'{query}\' and {active} given the evidence {set(evidence)}.')
        else:
            print(f'Active trails between \'{query}\' and {active} given no evidence.')
    else:
        print(f'No active trails for \'{query}\' given the evidence {set(evidence)}.')

active_trails_of("blueGoldDiff", ["blueWins", "towersDiff"])



def markov_blanket_of(node):
    print(f'Markov blanket of \'{node}\' is {set(model.get_markov_blanket(node))}')

markov_blanket_of(node='blueGoldDiff')

Active trails between 'blueGoldDiff' and {'blueExperienceDiff', 'killsDiff', 'minionsDiff', 'dragonsDiff'} given the evidence {'blueWins', 'towersDiff'}.
Markov blanket of 'blueGoldDiff' is {'blueWins', 'towersDiff', 'blueExperienceDiff', 'killsDiff', 'minionsDiff', 'dragonsDiff'}
